# Outlier Detection

In [ ]:
#Importing useful modules

import pandas as pd
from scipy.stats import zscore
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

## Loading and Cleaning Data

In [ ]:
# Reading and cleaning data
df = pd.read_excel('Dataset.xlsx')

new_columns = []
for col in df.columns:
    new_columns.append(col.replace(' ', '_').lower())
df.columns = new_columns
df = df.drop(columns=['quantity','day_supply','drug_ndc','pharmacy_id'])
num_claims = df.groupby('member_number')['claim_id'].count().reset_index()
df = df.merge(num_claims, on='member_number').rename(columns={'claim_id_y':'num_claims', 'claim_id_x':'claim_id'})

print(f'Shape of the dataset is {df.shape}')
df.head()

## Functions to check outliers

I created 3 different functions that needs Drug Class and one of the 3 costs and will return the maximum zscore within that sub-distribution if it is greater than 3 standard deviations from the mean. It will also return a dataframe of the potential outlier claims.

In [ ]:
# Function to check for outliers within Drug Class
def check_outliers_class(type_class, cost):

    test = df[df['drug_class'] == type_class]
    x = test[cost].values.reshape(-1,1)
    test['gc_outlier'] = abs(zscore(x))
    
    if test["gc_outlier"].max() > 3:
        print(f'{type_class}: \tMax score {test["gc_outlier"].max()}')
    
    num = len(test[test['gc_outlier'] > 3])
    outliers = test[test['gc_outlier'] > 3]

    return outliers

In [ ]:
# Function to check for outliers within Drug Name
def check_outliers_drug(type_drug, cost):

    test = df[df['drug_name'] == type_drug]
    x = test[cost].values.reshape(-1,1)
    test['gc_outlier'] = abs(zscore(x))
    
    if test["gc_outlier"].max() > 3:
        print(f'{type_drug}: \tMax score {test["gc_outlier"].max()}')
    
    num = len(test[test['gc_outlier'] > 3])
    outliers = test[test['gc_outlier'] > 3]

    return outliers

In [ ]:
# Function to check for outliers within Pharmacy Name
def check_outliers_pharm(type_pharm, cost):

    test = df[df['pharmacy_name'] == type_pharm]
    x = test[cost].values.reshape(-1,1)
    test['gc_outlier'] = abs(zscore(x))
    
    if test["gc_outlier"].max() > 3:
        print(f'{type_pharm}: \tMax score {test["gc_outlier"].max()}')
    
    num = len(test[test['gc_outlier'] > 3])
    outliers = test[test['gc_outlier'] > 3]

    return outliers

## Checking for outliers within Class

First, I will look for outliers within groups of the data. The first group is the Drug Class.

Found 8 total outliers within this group.

In [ ]:
#List of Drug Class values
drug_class_list = list(df.drug_class.unique())

#Loop that goes through the list of classes checking for outliers within group for Plan Cost
print('Based on Plan Cost')
for x in drug_class_list:
    test=x
    cost='plan_cost'
    check_outliers_class(test, cost)

#Loop that goes through the list of classes checking for outliers within group for Member Cost
print('\nBased on Member Cost')
for x in drug_class_list:
    test=x
    cost='member_cost'
    check_outliers_class(test, cost)

#Loop that goes through the list of classes checking for outliers within group for Gross Cost
print('\nBased on Gross Cost')
for x in drug_class_list:
    test=x
    cost='gross_cost'
    check_outliers_class(test, cost)

### Gross Cost

For drug classes, only two classes have outliers within their distribution.
In the Cancer class I found 2 outliers and within the Pain Management class I found 6 outliers.

It would be interesting to possibly look more into these 6 outliers for Pain Management

In [ ]:
class_outlier1 = 'CANCER'
class_outlier2 = 'PAIN MANAGEMENT'

In [ ]:
check_outliers_class(class_outlier1, 'gross_cost')

In [ ]:
check_outliers_class(class_outlier2, 'gross_cost')

### Plan Cost

Looking through classes based of plan cost, the same claims were detected as outliers (2 from Cancer class and 5 from Pain Management).

There were no outliers found based on the member cost for each class. 

In [ ]:
class_outlier1 = 'CANCER'
class_outlier2 = 'PAIN MANAGEMENT'

In [ ]:
check_outliers_class(class_outlier1, 'plan_cost')

In [ ]:
check_outliers_class(class_outlier2, 'plan_cost')

## Checking for outliers within drug

Now I will look at each drug by the different costs.

Found 6 total outliers within this group

In [ ]:
#List of Drug Name values
drug_list = list(df.drug_name.unique())

#Loop that goes through the list of classes checking for outliers within group for Plan Cost
print('Based on Plan Cost')
for x in drug_list:
    test=x
    cost='plan_cost'
    check_outliers_drug(test, cost)

#Loop that goes through the list of classes checking for outliers within group for Member Cost
print('\nBased on Member Cost')
for x in drug_list:
    test=x
    cost='member_cost'
    check_outliers_drug(test, cost)

#Loop that goes through the list of classes checking for outliers within group for Gross Cost
print('\nBased on Gross Cost')
for x in drug_list:
    test=x
    cost='gross_cost'
    check_outliers_drug(test, cost)

### Member Cost

There was 1 outlier within Cancer Drug 1 claims. For Cancer Drug 4, there were 2 outliers.

In [ ]:
drug_outlier1 = 'Cancer Drug 1'
drug_outlier4 = 'Cancer Drug 4'

In [ ]:
check_outliers_drug(drug_outlier1, 'member_cost')

In [ ]:
check_outliers_drug(drug_outlier4, 'member_cost')

### Gross Cost

Looking at the different drugs by gross cost, all 4 Cancer drugs had 1 claim that was flagged. 3 of the 4 claims have already been found from analysis above. 

In [ ]:
drug_outlier1 = 'Cancer Drug 1'
drug_outlier2 = 'Cancer Drug 2'
drug_outlier3 = 'Cancer Drug 3'
drug_outlier4 = 'Cancer Drug 4'

In [ ]:
check_outliers_drug(drug_outlier1, 'gross_cost')

In [ ]:
check_outliers_drug(drug_outlier2, 'gross_cost')

In [ ]:
check_outliers_drug(drug_outlier3, 'gross_cost')

In [ ]:
check_outliers_drug(drug_outlier4, 'gross_cost')

### Plan Cost

The same claims were flagged when looking at plan cost than what was just found looking at gross cost.

In [ ]:
drug_outlier1 = 'Cancer Drug 1'
drug_outlier2 = 'Cancer Drug 2'
drug_outlier3 = 'Cancer Drug 3'
drug_outlier4 = 'Cancer Drug 4'

In [ ]:
check_outliers_drug(drug_outlier1, 'plan_cost')

In [ ]:
check_outliers_drug(drug_outlier2, 'plan_cost')

In [ ]:
check_outliers_drug(drug_outlier3, 'plan_cost')

In [ ]:
check_outliers_drug(drug_outlier4, 'plan_cost')

## Checking for outliers within pharmacy

Lastly, I will look through each pharmacy to search for unusual payments.

I found 46 outliers within this group

In [ ]:
#List of Drug Name values
pharmacy_list = list(df.pharmacy_name.unique())

#Loop that goes through the list of classes checking for outliers within group for Plan Cost
print('Based on Plan Cost')
for x in pharmacy_list:
    test=x
    cost='plan_cost'
    check_outliers_pharm(test, cost)

#Loop that goes through the list of classes checking for outliers within group for Member Cost
print('\nBased on Member Cost')
for x in pharmacy_list:
    test=x
    cost='member_cost'
    check_outliers_pharm(test, cost)

#Loop that goes through the list of classes checking for outliers within group for Gross Cost
print('\nBased on Gross Cost')
for x in pharmacy_list:
    test=x
    cost='gross_cost'
    check_outliers_pharm(test, cost)

### Member Cost

By member cost, I found 28 claims within 5 of the 6 pharmacy were possible outliers.

In [ ]:
pharm_outlier1 = 'CVS'
pharm_outlier2 = 'UAB Pharmacy'
pharm_outlier3 = 'WALMART'
pharm_outlier4 = 'PUBLIX'
pharm_outlier5 = 'WALGREENS'

In [ ]:
check_outliers_pharm(pharm_outlier1, 'member_cost')

In [ ]:
check_outliers_pharm(pharm_outlier2, 'member_cost')

In [ ]:
check_outliers_pharm(pharm_outlier3, 'member_cost')

In [ ]:
check_outliers_pharm(pharm_outlier4, 'member_cost')

In [ ]:
check_outliers_pharm(pharm_outlier5, 'member_cost')

### Gross Cost

By gross cost, I found 23 claims with each pharmacy having at least one outlier. There are a couple claims that were found in the analysis by member cost right before this. 

In [ ]:
pharm_outlier1 = 'CVS'
pharm_outlier2 = 'UAB Pharmacy'
pharm_outlier3 = 'RX 4 U'
pharm_outlier4 = 'WALMART'
pharm_outlier5 = 'PUBLIX'
pharm_outlier6 = 'WALGREENS'

In [ ]:
check_outliers_pharm(pharm_outlier1, 'gross_cost')

In [ ]:
check_outliers_pharm(pharm_outlier2, 'gross_cost')

In [ ]:
check_outliers_pharm(pharm_outlier3, 'gross_cost')

In [ ]:
check_outliers_pharm(pharm_outlier4, 'gross_cost')

In [ ]:
check_outliers_pharm(pharm_outlier5, 'gross_cost')

In [ ]:
check_outliers_pharm(pharm_outlier6, 'gross_cost')

### Plan Cost

The last group found to have the same 23 claims flagged from that of gross cost. 

In [ ]:
pharm_outlier1 = 'CVS'
pharm_outlier2 = 'UAB Pharmacy'
pharm_outlier3 = 'RX 4 U'
pharm_outlier4 = 'WALMART'
pharm_outlier5 = 'PUBLIX'
pharm_outlier6 = 'WALGREENS'

In [ ]:
check_outliers_pharm(pharm_outlier1, 'plan_cost')

In [ ]:
check_outliers_pharm(pharm_outlier2, 'plan_cost')

In [ ]:
check_outliers_pharm(pharm_outlier3, 'plan_cost')

In [ ]:
check_outliers_pharm(pharm_outlier4, 'plan_cost')

In [ ]:
check_outliers_pharm(pharm_outlier5, 'plan_cost')

In [ ]:
check_outliers_pharm(pharm_outlier6, 'plan_cost')

## Potential Outliers

In total there are 57 claims (5.7%) that were flagged within the 3 groups analyzed from. Now I will look at the total distribution for each cost and find whether these 57 claims are also outliers of the entire distribution. I will continue to use gross, plan, and member cost. 

In [ ]:
po = [41,49,252,255,257,552,558,40,329,429,924,43,44,46,51,53,56,57,59,223,225,226,228,321,322,324,
      327,328,420,422,425,426,428,520,522,524,621,623,624,625,628,720,722,820,822,825,827,829,921,
      923,926,928,528,559,726,727,729]

x = df['gross_cost'].values.reshape(-1,1)
y = df['plan_cost'].values.reshape(-1,1)
z = df['member_cost'].values.reshape(-1,1)
df['gc_outlier'] = abs(zscore(x))
df['pc_outlier'] = abs(zscore(y))
df['mc_outlier'] = abs(zscore(z))

df['member_prop'] = df['member_cost'] / df['gross_cost']

outliers = df.loc[po]

After running this, our outlier list has been condensed to 29 claims. All 29 claims are under the drug class for Cancer and vary through all 4 cancer drugs and all 6 pharmacies. 

I would definitely take a deeper look at 3 specific claims as they were outliers to all 3 cost variables. These 3 claims had gross_cost values of 85,000, 60,000, and 30,000 US dollars, which are the highest 3 values by over two times the amount of the next largest. It is interesting that each claim happened once each month and were all received from RX 4 U. 

The other 26 claims were found to be unusual in regards to member cost. This looks to be specifically because members paid over $1,000 in cost for those claims. For those 26 claims, the member cost was over 25% of the total gross cost of the claims, which is drastically higher to the average around 11% for all claims.  

### Main Outliers

In [ ]:
ml_outliers = outliers[outliers['gc_outlier'] > 3]
ml_outliers.head()

### Potential Outliers

In [ ]:
potential_outliers = outliers[outliers['mc_outlier'] > 3]
num = len(potential_outliers)
print(f'Number of total potential outliers - {num}')

### More Examination

These 6 outliers were from 2 different members. They were all Pain Drug 2 from Walmart and cost 5600 US Dollars total and only 20 for members. The average for the Pain Management Class is 528 for gross and 20 for members. The large gap of these claims present interesting statistics for further examination

In [ ]:
outliers[outliers['drug_class'] == 'PAIN MANAGEMENT']